# Advent of Code 2024

Let's try out ocaml this year!  I read a book about Standard ML like 20 years ago, but remember very little.  So there will be a learning curve, but it shouldn't be as bad as APL.

Is this thing on?

In [1]:
2+2;;

- : int = 4


## Reusable utilities

This section is for stuff I get tired of copying and pasting.

### Reading stuff from files

We seem to have to read stuff from files a lot.

In [2]:
let read_numbers_from_file file_name =
  let file = open_in file_name in
  let output = ref [] in
  try
    while true do
      let line = input_line file
               |> Str.split (Str.regexp " +")
               |> List.map int_of_string in
      output := line :: !output
    done;
    assert false
  with
    End_of_file -> List.rev !output

val read_numbers_from_file : string -> int list list = <fun>


## Day 1

[Link](https://adventofcode.com/2024/day/1)

### Part 1

They want me to read two columns from a file, sort them, and sum the differences.

In [3]:
let read_pairs_from_file file_name =
  read_numbers_from_file file_name
  |> List.map (fun (numbers) -> match numbers with
               | [x; y] -> (x, y)
               | _ -> failwith "expecting pair")

val read_pairs_from_file : string -> (int * int) list = <fun>


In [4]:
read_pairs_from_file "day1_example.txt"

- : (int * int) list = [(3, 4); (4, 3); (2, 5); (1, 3); (3, 9); (3, 3)]


Ok, that was a little ugly, but it could be worse.  Let's use some of this great type matching stuff.

In [5]:
let rec unzip pairs =
  match pairs with
  | [] -> ([], [])
  | (x, y) :: rest ->
    let (xs, ys) = unzip rest in
      (x::xs, y::ys)

val unzip : ('a * 'b) list -> 'a list * 'b list = <fun>


In [6]:
unzip([(1,2); (3,4); (5,6)])

- : int list * int list = ([1; 3; 5], [2; 4; 6])


That was fun.  But there's a library for this.

In [7]:
List.split [(1,2); (3,4); (5, 6)]

- : int list * int list = ([1; 3; 5], [2; 4; 6])


In [8]:
let day1_part1 pairs =
  let (xs, ys) = List.split pairs in
  List.combine (List.sort compare xs) (List.sort compare ys)
  |> List.map (fun (x, y) -> abs(x - y))
  |> List.fold_left (+) 0

val day1_part1 : (int * int) list -> int = <fun>


That is actually kind of pretty, although the syntax is meh.

In [9]:
read_pairs_from_file "day1_example.txt" |> day1_part1

- : int = 11


In [10]:
read_pairs_from_file "day1_input.txt" |> day1_part1

- : int = 1666427


### Part 2

Now we've got to count stuff.  There is no need to be especially clever about this.

In [11]:
let day1_part2 pairs =
  let (xs, ys) = List.split pairs in
  let count x = List.filter ((=) x) ys |> List.length in
    xs
    |> List.map (fun x -> x * (count x))
    |> List.fold_left (+) 0

val day1_part2 : (int * int) list -> int = <fun>


In [12]:
read_pairs_from_file "day1_input.txt" |> day1_part2

- : int = 24316233


## Day 2

[Link](https://adventofcode.com/2024/day/2)

### Part 1

Today we have to take adjacent differences of things.

In [13]:
let rec differences list =
  match list with
  | [] -> []
  | [x] -> [] 
  | x :: y :: rest -> x - y :: differences (y :: rest)

val differences : int list -> int list = <fun>


In [14]:
differences [7; 6; 4; 2; 1]

- : int list = [1; 2; 2; 1]


I kind of wish this were point free?  Like I wanted to write `increasing || decreasing`.  Oh well.

In [15]:
let safe levels =
  let decreasing = List.for_all (fun(x) -> x < 0) in
  let increasing = List.for_all (fun(x) -> x > 0) in
  let in_range = List.for_all (fun(x) -> abs(x) >= 1 && abs(x) <= 3) in
  let check xs = (increasing xs || decreasing xs) && in_range xs in
    check (differences levels)

let count safety_check data =
  data
  |> List.filter safety_check
  |> List.length

val safe : int list -> bool = <fun>


val count : ('a -> bool) -> 'a list -> int = <fun>


In [16]:
read_numbers_from_file "day2_example.txt" |> count safe

- : int = 2


In [17]:
read_numbers_from_file "day2_input.txt" |> count safe

- : int = 442


### Part 2

Now we have to try fudging the data, I guess.

In [18]:
let rec ablate xs =
  match xs with
  | [] -> []
  | x :: rest -> rest :: (List.map (fun(xs) -> x :: xs)) (ablate rest)

val ablate : 'a list -> 'a list list = <fun>


In [19]:
ablate [1;2;3;4]

- : int list list = [[2; 3; 4]; [1; 3; 4]; [1; 2; 4]; [1; 2; 3]]


In [20]:
let safeish levels =
  safe levels ||
  (ablate levels |> List.exists safe)

val safeish : int list -> bool = <fun>


In [21]:
read_numbers_from_file "day2_example.txt" |> count safeish

- : int = 4


In [22]:
read_numbers_from_file "day2_input.txt" |> count safeish

- : int = 493
